In [55]:
import json
import re
import pandas as pd
import openpyxl
from pandas.io.json import json_normalize

In [56]:
r = re.compile(r'[^\x0A\x20-\x7D]')
#katas = ['kowe', 'kanca', ' kuwi', ' iki ', ' kabeh', ' iku', ' mangan', ' krungu', 'melu ', ' koe', 'rodo', 'mbok', 'mergo']
#kata yang kira-kira bisa menjadi bagian dari kata lain bisa ditambahkan spasi
#misal 'iki' dalam 'dikita' 

In [57]:
with open('Draft_SMS_UGM.json') as f:
    d = json.load(f)
    for data in d['feeds']:
        data['content'] = r.sub('', data['content'].lower())
        temp = map(lambda x: r.sub('', x).replace('\n', ' ') if (x != None) else None, data['comments'])
        data['comments'] = "\n".join(filter(lambda x: x != None and x != '', temp))
        data['comments'] = data['comments'].lower()

In [58]:
df = json_normalize(d['feeds'])

In [59]:
df.set_index('postId', inplace = True)

In [60]:
ddf = df.content.str.extract(r'(?:\s*(?:dari|from)\s*:\s*(?P<dari>.+)\s+)?(?:(?:teruntuk|untuk|for|buat|kepada|ke)\s*:\s*(?P<untuk>.+)\s+)?(?:(?:pesan|messages?)?\s*:?\s*(?P<pesan>[\x00-\x7f]+))', flags=re.IGNORECASE)

In [61]:
df['dari'] = ddf['dari']
df['untuk'] = ddf['untuk']
df['pesan'] = ddf.pesan.str.replace('\n', ' ').replace(' {2,}', ' ', regex=True)
df['panjangPesanKata'] = 0
df['panjangPesanHuruf'] = 0
df['banyakKomentar'] = 0

In [62]:
df.drop(columns=['content'], inplace = True)

In [63]:
df.drop_duplicates(subset ="pesan",keep = False, inplace = True)

In [64]:
df.dropna(axis=0, how='any', inplace=True)

In [65]:
for index, row in df.iterrows():
    if(len(row['pesan'].split(' ')) < 5):
        df.drop(index, inplace=True)
    else:
        df.at[index, 'panjangPesanHuruf'] = len(row['pesan'])
        df.at[index, 'panjangPesanKata'] = len(row['pesan'].split(' '))
        df.at[index, 'banyakKomentar'] = len(row['comments'].split())
    
#    else:
#        for kata in katas:
#            if((kata in row['pesan'].lower()) or (kata in row['dari'].lower()) or (kata in row['untuk'].lower())):
#                df.drop(index, inplace=True)
#                break
                

In [66]:
df.head(5)

,comments,likeCount,dari,untuk,pesan,panjangPesanKata,panjangPesanHuruf,banyakKomentar
postId,,,,,,,,
1155040072910043061,iya ;( jd kangen masa masa ospek\naamiin\nhehe...,190,adik yang tidak pandai berbicara namun lebih m...,kakak tingkat favorit,"sering kali kami mengeluh tentang kalian ""mas ...",255,1774,31
1155066024410041180,sebelah mana nih\neh iya po? :((\nsebelah mana...,63,saya yang baru tau,kalian yang mau tau maupun udah tau,"ternyata jalan pogung di tutup ya :""",7,36,57
1155031044610047449,ngga lolos syarat kedua \n#ask kenapa ya donor...,119,wanita berpikiran konservatif,warga jogja,yuk kita berbuat baik. semoga perbuatan baik m...,11,69,99
1155039237710047873,yongono iku jenenge konco\nkeplak ae nder\nsam...,268,aku,koe,"he, aku pas dijaluki tulung opo ae ro koe yo a...",24,129,107
1155063528410048781,"kejatuhan ijo , jadi bangjo lu nder wkwk\nbera...",156,aku yang udah kejatuhan 2x,warga yujiem dan sekitarnya,aku mau kasih info aja nih gaes ya sedikit cer...,75,464,233


In [13]:
df.to_excel("Clean.xlsx")  # doctest: +SKIP